In [72]:
import cv2
import numpy as np

In [73]:
def ellipse_detection(img):
  
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (7,7), 20)   


  circles = cv2.HoughCircles(blur,cv2.HOUGH_GRADIENT,.5, 30,
                              param1=130,param2=70,minRadius=3)
  center = []

  if circles is not None:
    circles = np.uint16(np.around(circles))
    for i in circles[0,:]:
      center.append((i[0],i[1]))

  return center

In [74]:
def get_contours(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7,7), 20)

    edged = cv2.Canny(blur,40,100,L2gradient=True)

    structuringElement = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    dilated = cv2.morphologyEx(edged, cv2.MORPH_DILATE, structuringElement)

    contours, _ = cv2.findContours(dilated,cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    return contours

In [75]:
def get_contour_center(cnt):
    
    M = cv2.moments(cnt)
    cX = int(M['m10'] / M['m00'])
    cY = int(M['m01'] / M['m00'])

    return (cX,cY)

In [76]:
def get_k_nearest(poly, point, k=4):
    (x,y) = point
    return sorted(poly, key=lambda pt: ((pt[0][0] - x)**2 + (pt[0][1] - y)**2))[:k]

In [77]:
def cross_detection(image):

    contours = get_contours(image)

    cross_candidates = []

    for cnt in contours:

        perimeter = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.01*perimeter, True)

        hull = cv2.convexHull(cnt, returnPoints=False)
        defects = cv2.convexityDefects(cnt, hull)

        if defects is None:
            continue
        
        if (8 <= len(approx) <= 14) and (cv2.isContourConvex(cnt) == False):
            
            center = get_contour_center(cnt)
            four_nearest = get_k_nearest(approx, center)

            cross_candidates.append((center, four_nearest))

    return cross_candidates


In [78]:
def landing_pad_detection(image):
    
    ellipse_candidates = ellipse_detection(image)
    cross_candidates = cross_detection(image)

    min_dist = 1e7
    points = []
    center = []

    for ellipse_center in ellipse_candidates:
        for (cross_center, points_) in cross_candidates:
            dist = (ellipse_center[0] - cross_center[0])**2 + (ellipse_center[1] - cross_center[1])**2
            if dist < min_dist:
                min_dist = dist
                points = points_
                center = cross_center

    if len(points) == 0:
        return False, [], []

    return True, points, center

In [79]:
matriz_intrinseca = np.array([[3.0214060e+02,   0.0000000e+00,   0.0000000e+00],
                              [0.0000000e+00,   3.0202364e+02 ,  0.0000000e+00],
                              [1.5870089e+02,   1.1872175e+02,   1.0000000e+00]])

In [80]:
def homogeneo_para_cartesiano(vetor_coord_homogenea):
    vetor = vetor_coord_homogenea[:-1,:] / vetor_coord_homogenea[-1,0]
    return vetor

In [81]:
def cartesiano_para_homogeneo(vetor):
    vetor_coord_homogenea = np.append(vetor, np.array([[1]]), 0)
    return vetor_coord_homogenea

In [82]:
def transformacao_projetiva_3d(ponto_projetado_2d_homogeneo, matriz_intrinseca):
    ponto3d = np.linalg.inv(matriz_intrinseca) @ ponto_projetado_2d_homogeneo
    return ponto3d

In [83]:
def pixels2meters(arr):
    return arr * 0.0013633547

In [85]:
image = cv2.imread('image/landing_pad_1.png')
final_image = cv2.imread('image/landing_pad_1.png')

ret, points, center = landing_pad_detection(image)

if ret:
    for pt in points:
        cv2.circle(final_image, (pt[0][0], pt[0][1]), 7, (255, 255, 255), -1)

cv2.circle(final_image, (center[0], center[1]), 7, (0, 0, 255), -1)

center = np.array(center).reshape(1,2)
c_homogeneo = cartesiano_para_homogeneo(center.T)
center_3d_pixel = transformacao_projetiva_3d(c_homogeneo, matriz_intrinseca)
center_3d = pixels2meters(center_3d_pixel)
dst2center = ((center_3d[0][0])**2 + (center_3d[1][0])**2 + (center_3d[2][0])**2)**0.5

dst_str = 'DST TO CENTER = ' + '%.*f' % (2, dst2center) + 'm'
final_image =  cv2.putText(final_image, dst_str, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0,0,255), 2, cv2.LINE_AA)

cv2.imshow('initial_image', image)
cv2.imshow('final_image', final_image)
cv2.waitKey(0)


-1

In [91]:
cap = cv2.VideoCapture('video/landing_pad.avi')
 
if (cap.isOpened()== False): 
    print("Error opening video stream or file")
 
while(cap.isOpened()):
    ret, image = cap.read()
    final_image = image.copy()
    if ret == True:
        retval, points, center = landing_pad_detection(image)

        if not retval:
            continue

        for pt in points:
    
            cv2.circle(final_image, (pt[0][0], pt[0][1]), 7, (255, 255, 255), -1)

        cv2.circle(final_image, (center[0], center[1]), 7, (0, 0, 255), -1)

        center = np.array(center).reshape(1,2)
        c_homogeneo = cartesiano_para_homogeneo(center.T)
        center_3d_pixel = transformacao_projetiva_3d(c_homogeneo, matriz_intrinseca)
        center_3d = pixels2meters(center_3d_pixel)
        dst2center = ((center_3d[0][0])**2 + (center_3d[1][0])**2 + (center_3d[2][0])**2)**0.5

        dst_str = 'DST TO CENTER = ' + '%.*f' % (2, dst2center) + 'm'
        final_image =  cv2.putText(final_image, dst_str, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (0,0,255), 2, cv2.LINE_AA)

        cv2.imshow('initial_image', image)
        cv2.imshow('final_image', final_image)
    

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
 
    else: 
        break
 
cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'copy'